# Classificador de Produtos com GPT-3 - Curie



---

In [1]:
import csv
from dotenv import load_dotenv
import openai
import os
import pandas as pd
import time

> Warning: OpenAI pode cobrar por chamadas à API após um determinado número de solicitações, portanto, certifique-se de ler e entender os termos e condições antes de começar a usar a API.

In [2]:
load_dotenv()  # Carrega as variáveis do arquivo .env
openai.api_key = os.getenv('OPENAI_API_KEY')

As engines davinci, curie e babbage são os três modelos mais poderosos disponíveis na plataforma OpenAI atualmente. Cada um desses modelos tem capacidades diferentes e é recomendado para diferentes tipos de tarefas.

- Davinci: é a engine mais poderosa da OpenAI atualmente. Ela é capaz de lidar com uma ampla variedade de tarefas, incluindo geração de linguagem natural, tradução, resumo de texto, análise de sentimentos e muito mais. É a engine recomendada para tarefas de alta complexidade, em que é necessário obter a melhor qualidade possível.


- Curie: é uma engine intermediária, que oferece uma combinação equilibrada de potência e eficiência. Ela é recomendada para tarefas que exigem uma capacidade razoável, mas não precisam da potência total da engine Davinci.


- Babbage: é a engine mais leve e econômica da OpenAI. Ela é recomendada para tarefas que exigem uma capacidade mais simples e econômica, como a classificação de texto e a análise de sentimentos básica.

> Nesse notebook estamos utilizando a engine `curie`.

In [3]:
# Função para extrair informações de marca, volume, embalagem e unidade a partir de uma descrição de produto
def extract_product_info(description):
    response = openai.Completion.create(
      engine="curie",
      prompt=f"Classifique as informações do produto: {description}\nMarca:\nVolume:\nEmbalagem:\nUnidade:",
      max_tokens=100,
      n=1,
      stop=None,
      temperature=0.5,
    )
    output = response.choices[0].text.strip().split('\n')
    info_dict = {'marca': output[0], 'volume': output[1], 'embalagem': output[2], 'unidade': output[3]}
    return info_dict

In [4]:
# data = pd.read_excel('catalogo-produtos-grupos-validado.xlsx')
# data.info()

In [11]:
# prod_list = data[['DSC_PRODUTO']]
# # Seleciona uma amostra aleatória de 10% das linhas do DataFrame
# amostra = prod_list.sample(frac=0.01)
# amostra.to_csv('lista_produtos_sample.csv', index=False)

In [6]:
# Carregue o dataframe com as descrições de produtos
df = pd.read_csv('lista_produtos_sample.csv')

In [10]:
df

,DSC_PRODUTO
0,KUAT PET 2L
1,CERV PETRA PURA MALTE GRF RET 600ML CX24
2,GATORADE LIMAO PET 500ML SIX PACK
3,REF SCHWEPPES CITRUS
4,GUARANA CHP ANTARCTICA PET 200ML SH C/12 (PROM...
...,...
109,BIG ENERGY 01X2000ML QTD. 12.00 UN
110,AM CRYS BR PET 350ML S/G
111,AG MIN CRYSTAL C G 1
112,SODA LIMONADA ANTARCTICA PET 200ML SH C/12


In [13]:
# Crie um novo dataframe para armazenar as informações de cada produto
new_df = pd.DataFrame(columns=['DSC_PRODUTO', 'marca', 'volume', 'embalagem', 'unidade'])
new_df

,DSC_PRODUTO,marca,volume,embalagem,unidade


In [16]:
import re

def find_first_number(tokens):
    """
    Encontra a posição do primeiro número na lista de tokens.
    Retorna -1 se não encontrar nenhum número.
    """
    for i, token in enumerate(tokens):
        if re.match(r'^\d', token):
            return i
    return -1

In [17]:
def extract_product_info(description):
    info_dict = {
        'marca': '',
        'volume': '',
        'embalagem': '',
        'unidade': ''
    }
    
    # Tokenize a descrição
    tokens = description.split()
    
    # Encontra a posição do primeiro número na descrição
    num_pos = find_first_number(tokens)
    
    # Extrai a marca
    if num_pos > 1:
        info_dict['marca'] = ' '.join(tokens[:num_pos-1])
    
    # Extrai o volume e a embalagem
    if num_pos > 0:
        volume_tokens = tokens[num_pos:]
        for token in volume_tokens:
            if token.endswith('ML') or token.endswith('L'):
                info_dict['volume'] = token
            elif token.startswith('LT'):
                info_dict['embalagem'] = token
    
    # Extrai a unidade
    unidade = re.findall('\d+(?:\.\d+)?\s*(KG|G|ML|L|UN)', description.upper())
    if unidade and len(unidade) > 0:
        info_dict['unidade'] = unidade[0]
    
    return info_dict

In [18]:
# Loop para iterar sobre as descrições de produtos e extrair as informações de cada um
for index, row in df.iterrows():
    description = row['DSC_PRODUTO']
    info_dict = extract_product_info(description)
    new_df.loc[index] = [description, info_dict['marca'], info_dict['volume'], info_dict['embalagem'], info_dict['unidade']]

In [19]:
new_df

,DSC_PRODUTO,marca,volume,embalagem,unidade
0,KUAT PET 2L,KUAT,2L,,L
1,CERV PETRA PURA MALTE GRF RET 600ML CX24,CERV PETRA PURA MALTE GRF,600ML,,ML
2,GATORADE LIMAO PET 500ML SIX PACK,GATORADE LIMAO,500ML,,ML
3,REF SCHWEPPES CITRUS,,,,
4,GUARANA CHP ANTARCTICA PET 200ML SH C/12 (PROM...,GUARANA CHP ANTARCTICA,200ML,,ML
...,...,...,...,...,...
109,BIG ENERGY 01X2000ML QTD. 12.00 UN,BIG,01X2000ML,,ML
110,AM CRYS BR PET 350ML S/G,AM CRYS BR,350ML,,ML
111,AG MIN CRYSTAL C G 1,AG MIN CRYSTAL C,,,
112,SODA LIMONADA ANTARCTICA PET 200ML SH C/12,SODA LIMONADA ANTARCTICA,200ML,,ML


> Esse notebook apresentou muitos problemas na classificação. Vamos para o próximo!

In [17]:
# Salva em um arquivo Excel
# new_df.to_excel('produtos-classificados-curie.xlsx', index=False)